In [36]:
import pickle

with open('./datasets/small/embeddings/e_H.pickle', 'rb') as f:
    e_H = pickle.load(f)
with open('./datasets/small/embeddings/e_A.pickle', 'rb') as f:
    e_A = pickle.load(f)
with open('./datasets/small/embeddings/e_S.pickle', 'rb') as f:
    e_S = pickle.load(f)

# LDA

In [8]:
from time import time

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

def lda(embedding):
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=embedding['corpus'],
            id2word=embedding['id2word'],
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
            random_state=99
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=embedding['T'], dictionary=embedding['id2word'], coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, embedding['id2word']))
        return (cs + ts) / 2

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)
    t_end = time() # TODO: this should be after best_lda

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    return {
        'best_params': study.best_params,
        'best_trial': best_trial,
        'time': t_end - t_start,
        'model': best_lda,
    }

In [9]:
def save_models_lda(embeddings, embedding_names):
    for name in embedding_names:
        model = lda(embeddings[name])
        with open(f'./datasets/small/models_lda/{name}.pickle', 'wb') as f:
            pickle.dump(model, f, protocol=pickle.HIGHEST_PROTOCOL)
        del model

In [10]:
e_variant = ['T', 'C', 'CL', 'CLW', 'CW', 'L', 'LW', 'W']

In [11]:
# This takes 1.5 hour
e_names = [f'H{t}' for t in e_variant]
# save_models_lda(e_H, e_names)
e_names = [f'A{t}' for t in e_variant]
# save_models_lda(e_A, e_names)
e_names = [f'S{t}' for t in e_variant]
# save_models_lda(e_S, e_names)

# BERTopic

In [33]:
import pandas as pd

H = pd.read_csv('./datasets/small/HCLW.csv')
A = pd.read_csv('./datasets/small/ACLW.csv')
S = pd.read_csv('./datasets/small/SCLW.csv')

In [59]:
from bertopic import BERTopic

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"
bertopic = BERTopic(embedding_model=embedding_model)


def save_models_bertopic(text_title, texts, variant_names):
    for v in variant_names:
        model = bertopic.fit(texts[v].dropna())
        model.save(
            f"./datasets/small/models_bertopic/{text_title}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        del model

In [58]:
e_variant = ['CW', 'CLW']
save_models_bertopic('H', H.sample(10), e_variant)
save_models_bertopic('A', A.sample(10), e_variant)
save_models_bertopic('S', S.sample(10), e_variant)